# RNNs

## RNNs and Language Models

- 循环神经网络和语言模型
  - 循环神经网络：通过隐藏状态来存储之前时间步的信息
    - 不含隐藏状态的神经网络（前馈NN）：
      - 对每个神经元，隐藏层的输出$H=\phi(XW_x+b_h)$
      - 输出层的输出为：$O=\phi(HW_h+b_o)$
    - 含隐藏状态的神经网络：考虑输入数据存在时间相关性的情况。保存上一时间步的隐藏变量$H_{t−1}$，时间步tt的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定。
      - 隐藏层：$H_t=\phi(X_tW_{xh}+H_{t-1}W_{hh}+b_h)$
      - 输出层：$O_t=\phi(H_tW_{ho}+b_o)$
  - 语言模型的任务：对数据输入，预测下一个字符。
    - 概率语言模型：$P(w_1,w_2,...,w_n)=\prod_{i=1}^nP(w_i|w_1,w_2,....w_{i-1})$
    - 用循环神经网络构建语言模型：用字符级循环神经网络（character-level recurrent neural network）构建更为简单，因为字符的种类要远远小于词或短语的种类。
  - 数据集：周杰伦歌词

In [112]:
import torch
import random
#import zipfile
# with zipfile.Zipfile(PATH) as a:
# with ...txt...
import d2lzh_pytorch as d2dl

### 预处理

- 一些细节
  - 代码封装在d2lzh_pytorch包里的`load_data_jay_lyrics`函数中，以方便后面章节调用。调用该函数后会依次得到corpus_indices、char_to_idx、idx_to_char和vocab_size这4个变量。
  - `vocab_size`为1027
  - `char_list`即idx_to_char: char的list，互不相同。知道index可在list查找char
  - `char_dict`即char_to_idx:char字典，互不相同，value是char在list中的特异下标。知道char可以在dict查找index
  - `corpus_indices`: corpus中每个词替换成在字典中对应的下标。

In [113]:
# Load Data
with open('jaychou_lyrics.txt') as f:
    corpus_chars = f.read()
print(corpus_chars[:40])
print(len(corpus_chars))

# Data Preprocessing:
# replace \n with ' '
corpus_chars = corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars = corpus_chars[:10000]
corpus_chars[:40]

想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每
63282


'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每'

In [114]:
# Build a char-index
#print(type(corpus_chars))  # str
char_list = list(set(corpus_chars))  # set(): make it distinct
char_dict = dict([(char,i) for i, char in enumerate(char_list)])
print(len(char_dict))

i=0
for key, value in char_dict.items():
    if i<5:
        print(key, value)
        i += 1

1027
呜 0
欣 1
哈 2
面 3
可 4


In [115]:
# transfer training data to index
corpus_indices = [char_dict[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:',''.join([char_list[idx] for idx in sample]))
print('indices:',sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [393, 887, 551, 985, 749, 96, 197, 393, 887, 512, 708, 302, 709, 336, 536, 103, 197, 393, 887, 512]


### 时序数据
- 时序数据一个样本是连续的字符，标签是这些字符在训练集中的下一个字符。
- 对时序数据采样（batch）的方法有
  - **随机采样**：其中批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。 在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。
    - `d2dl.data_iter_random`
  - **相邻采样**：令相邻的两个随机小批量在**原始序列**上的位置相毗邻。可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。
    - 好处：在训练模型时我们只需在该迭代周期开始时初始化隐藏层。
    - 坏处：若样本相邻，batch之间的隐藏层串联，梯度计算的时候将会彼此依赖，那么计算到后面的成本很大。
      - 解决方法：在batch之间将hidden的梯度从计算图中分离开来。
    - 具体构造：
      - 想象一列句子（由char或char的对应的indice组成），我们把它变成一个(batch_size,-1)的矩阵，这样每列是一个batch，列与列之间每行对应的char是连续（consecutive）的。
      - 再用列数除以一个样本所含的char数量num_steps，就是batch数量。
      - 最后最这个矩阵遍历获得iter的X和Y。（Y比X在位置上多1个char）
    - `d2dl.data_iter_consecutive`

In [116]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    num_examples = (len(corpus_indices) - 1) // num_steps  # one batch contains how many Xs, X: num_steps char, 
    num_batch = num_examples // batch_size  # one epoch contains how many batches
    example_indices = list(range(num_examples))  # [0,1,...exp_num]
    random.shuffle(example_indices)  # [3,6,1,...,]
    
    def _data(pos):
        return corpus_indices[pos:pos+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(num_batch):
        i = i * batch_size
        batch_indices = example_indices[i:i+batch_size]
        X = [_data(j*num_steps) for j in batch_indices]
        Y = [_data(j*num_steps+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device), torch.tensor(Y,dtype=torch.float32,device=device)
    
# test
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [ 6.,  7.,  8.,  9., 10., 11.]]) 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [ 7.,  8.,  9., 10., 11., 12.]]) 

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [12., 13., 14., 15., 16., 17.]]) 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [13., 14., 15., 16., 17., 18.]]) 



In [117]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    char_len = len(corpus_indices)
    # construct a matrix that, 
    # every row is a consecutive sentence(chars) and every column is a batch (not consecutive in inside a batch).
    length = char_len // batch_size 
    indices = corpus_indices[0:length*batch_size].view(batch_size,length) 
    batch_num = (length-1) // num_steps  # this is the batch_num to be consecutive
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+num_steps+1]
        yield X, Y

        
my_seq = list(range(30))
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n', 'X.size:', X.size())   

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]]) 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]]) 
 X.size: torch.Size([2, 6])
X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]]) 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]]) 
 X.size: torch.Size([2, 6])


In [118]:
a = torch.tensor([1,2,3,4,5,6])
print(a.view(2,3))

tensor([[1, 2, 3],
        [4, 5, 6]])


## RNN from Scratch

- 一些细节
  - 训练
    - RNN的outputs是num_step个(batch_size, vocab_size_one-hot)的列表，需要用torch.cat(dim=0)来将列表中元素按照第一维度(batch_size)进行拼接。
    - y是(num_batch, num_steps), value是一个(0,vocabsize)的值
    - 输入进crossEntropy比较的应该是（N\*C）于（N）。实际上RNN的输出的交叉熵就相当于一个vocab_size的多分类问题。
  - contiguous(): 在pytorch，view等方法约定不改变原内存，类似，用transpose等共享内存操作。所以如果我们想要按照这类操作之后的按行优先的tensor排列方式组织内存，需要contiguous来开辟一个新的内存来存储。
    - 或者，用reshape函数就行
    
  - argmax返回index

In [119]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import d2lzh_pytorch as d2dl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [120]:
# Load Data
def load_data_jay_lyrics():
    with open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size


(corpus_indices, char_dict, char_list, vocab_size) = load_data_jay_lyrics()

In [121]:
# one-hot method 1:
print(vocab_size)
inputs = F.one_hot(torch.LongTensor(corpus_indices), vocab_size)
print(inputs.size())

# one-hot method 2:
def one_hot_transform(x, n_class, dtype=torch.float32):
    # X is like tensor:[2,3,45,67765,...]
    x = x.long()
    res = torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

def one_hot_fit(x, n_class, dtype=torch.float32):
    # input x: (batch_size, seq_len(time_step))
    return [one_hot_transform(x[:,i],n_class,dtype) for i in range(x.shape[1])]

my_seq = list(range(30))
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    inputs = one_hot_fit(X, vocab_size)
print(inputs[0].size())

1027
torch.Size([10000, 1027])
torch.Size([2, 1027])


In [122]:
# Define Model
# initialization & hyperparameters
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    W_ih = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    W_ho = _one((num_hiddens,num_outputs))
    b_o = torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_ih,W_hh,b_h,W_ho,b_o])


# model
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

def rnn(inputs, state, params):
    # inputs含有num_step个(batch,vocab)
    W_ih, W_hh, b_h, W_ho, b_o = params
    H, = state
    outputs = []
    for X in inputs:  # 对inputs里每个time step算
        H = torch.tanh(torch.matmul(X,W_ih)+torch.matmul(H,W_hh)+b_h)
        Y = torch.matmul(H,W_ho)+b_o
        outputs.append(Y)
    return outputs, (H,)

def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta/norm)


In [123]:
prefix='分开'
print(prefix[0])
char_dict['分']

分


621

In [124]:
# Make Prediction: predict num_chars words based on the prefix
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size,\
                device, char_list, char_dict):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_dict[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        # input: previous output
        X = one_hot_fit(torch.tensor([[output[-1]]],device=device), vocab_size)
        # calculate the hidden state
        (Y,state) = rnn(X,state,params)
        # the predicted word is the argmax word
        if t < len(prefix)-1:
            output.append(char_dict[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(1).item()))  # argmax返回index
    return ''.join([char_list[i] for i in output])

# Train
def train_and_predict(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device,\
                     corpus_indices, char_list, char_dict, is_random_iter, num_epochs,\
                     num_steps, lr, clipping_theta, batch_size, pred_period, pred_len,\
                     prefixes):
    if is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
        data_iter_fn = d2dl.data_iter_random
    else:
        data_iter_fn = d2dl.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:   # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                # print(state[0].size())
                for s in state:
                    s.detach_()
            
            inputs = one_hot_fit(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵,每个时刻的hidden output。
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(batch1-batch2-batch3...batch_num, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # 变成（batch*num_step）的行向量，变成这样跟输出的行一一对应
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清零
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2dl.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            
        if (epoch+1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,\
                                       num_hiddens, vocab_size, device, char_list, char_dict))
                
    
        
        
    

In [125]:
# 用随机模型
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, char_list,
                      char_dict, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)



epoch 50, perplexity 69.205615, time 0.67 sec
 - 分开 我想要你想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我想要你想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 100, perplexity 9.802858, time 0.64 sec
 - 分开 我不要再想 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离不我 不知不觉 你已经这样我 
 - 不分开吗 我不能再想 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离不我 不知不觉 你已经这样我
epoch 150, perplexity 2.791066, time 0.65 sec
 - 分开 我 想带你骑单车 我 想和你看棒我的爸 就怎么 你给我都开你 有话去对医药箱说 别怪我 别怪我 说
 - 不分开吗 我不能爸 你打我妈 这样我遇多功猜样 别的让斑重比 再狠狠 我不的 如你开这我里是 但透想你 一
epoch 200, perplexity 1.580559, time 0.70 sec
 - 分开 一只到双拳  让它你着我都多难恼多烦恼  我过你烦 我有多烦恼  没有你在我有多难熬多烦恼  穿过
 - 不分开吗 然后将过去 慢慢布习 全后怕日出 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空
epoch 250, perplexity 1.299528, time 0.59 sec
 - 分开 一只伦双截 我妈往 如过我 是你怎么信代 有管是哪里都是晴 别怪我 别怪我 难你怎么旧每日折一枝杨
 - 不分开简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 周杰伦


In [126]:
# 用相邻模型
train_and_predict(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, char_list,
                      char_dict, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 65.438806, time 0.62 sec
 - 分开 我想要你 温小了双 如谁我有 你是我有 我想一空 如果我有 你谁一人 如果我有 你谁一人 如果我有
 - 不分开 我想要你 温小了双 如谁我有 你是我有 我想一空 如果我有 你谁一人 如果我有 你谁一人 如果我有
epoch 100, perplexity 7.433057, time 0.62 sec
 - 分开 我想就这样 别不人开口 仙人在怕羞 蜥蝪横人走 这里在么走 有蝪在人走 有里在么走 有底在人走 有
 - 不分开力 你天经离 我小了空  一场梦停 全暖了空  一话苦在 我有多烦熬 我不定很抽  没有你烦我有多烦
epoch 150, perplexity 2.142149, time 0.73 sec
 - 分开 我想无 爱情么著看着我 别发抖 快给我抬起头 有话去对医药箱说 别怪我 别怪我 说你  说 回有拳
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.299166, time 0.75 sec
 - 分开 问候我 谁是了枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 这故事 告诉我 印地安的传说 
 - 不分开觉我你 经堡 酒去为壁倒  教是一口 我给著努力向 景天入秋 漫脸黄沙落寞 近北情怯的我 相思寄红豆
epoch 250, perplexity 1.183328, time 0.79 sec
 - 分开 问候我 谁是一枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 一故事 告诉我遇 他止我一天活
 - 不分开觉 你已经离 我给我带 说隐出空 你一定梦 都有不同 你不懂 连一知轻重 泪水鲜红 也面放纵 恨自己
